In [1]:
import os
import glob

# Define root directory
ROOT_FOLDER = r"C:\Users\bmoha\Desktop\DataCollection"

# Find all heat_logs/2025-* subfolders (recursively)
heatlog_folders = glob.glob(os.path.join(ROOT_FOLDER, "*", "heat_logs", "2025-*"))

print(f"📁 Found {len(heatlog_folders)} folders under heat_logs:\n")

# Count and print files in each folder
for folder in heatlog_folders:
    all_files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    print(f"📂 {folder} — {len(all_files)} files")


📁 Found 18 folders under heat_logs:

📂 C:\Users\bmoha\Desktop\DataCollection\1060\heat_logs\2025-6-5_13161 — 66 files
📂 C:\Users\bmoha\Desktop\DataCollection\2150\heat_logs\2025-6-5_101933 — 1 files
📂 C:\Users\bmoha\Desktop\DataCollection\2150\heat_logs\2025-6-5_102122 — 1 files
📂 C:\Users\bmoha\Desktop\DataCollection\2150\heat_logs\2025-6-5_102239 — 6 files
📂 C:\Users\bmoha\Desktop\DataCollection\2150\heat_logs\2025-6-5_103840 — 48 files
📂 C:\Users\bmoha\Desktop\DataCollection\2182\heat_logs\2025-6-6_111420 — 60 files
📂 C:\Users\bmoha\Desktop\DataCollection\2226\heat_logs\2025-6-7_111516 — 28 files
📂 C:\Users\bmoha\Desktop\DataCollection\2226\heat_logs\2025-6-7_115325 — 19 files
📂 C:\Users\bmoha\Desktop\DataCollection\2462\heat_logs\2025-6-4_174114 — 65 files
📂 C:\Users\bmoha\Desktop\DataCollection\2973\heat_logs\2025-6-7_17131 — 32 files
📂 C:\Users\bmoha\Desktop\DataCollection\3154\heat_logs\2025-6-4_131612 — 30 files
📂 C:\Users\bmoha\Desktop\DataCollection\3154\heat_logs\2025-6-4_14

In [1]:
import os
from collections import defaultdict

# Change to your root directory
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"

# Store results grouped by participant ID
participants = defaultdict(lambda: {"total": 0, "sessions": {}})

# Traverse all folders
for dirpath, dirnames, filenames in os.walk(ROOT):
    if "heat_logs" in dirpath:
        parts = dirpath.split(os.sep)
        try:
            participant_id = parts[-3]  # e.g., '2150'
            session = parts[-1]         # e.g., '2025-6-5_13161'
            file_count = len([f for f in filenames if f.endswith('.csv') or f.endswith('.xlsx') or f.endswith('.json')])
            if file_count > 0:
                participants[participant_id]["total"] += file_count
                participants[participant_id]["sessions"][session] = file_count
        except IndexError:
            pass

# Print summary
for pid, data in sorted(participants.items()):
    print(f"\n👤 Participant {pid} — {len(data['sessions'])} sessions — {data['total']} files total")
    for session, count in sorted(data["sessions"].items()):
        print(f"    📂 {session} — {count} files")



👤 Participant 1060 — 1 sessions — 66 files total
    📂 2025-6-5_13161 — 66 files

👤 Participant 2150 — 4 sessions — 56 files total
    📂 2025-6-5_101933 — 1 files
    📂 2025-6-5_102122 — 1 files
    📂 2025-6-5_102239 — 6 files
    📂 2025-6-5_103840 — 48 files

👤 Participant 2182 — 1 sessions — 60 files total
    📂 2025-6-6_111420 — 60 files

👤 Participant 2226 — 2 sessions — 47 files total
    📂 2025-6-7_111516 — 28 files
    📂 2025-6-7_115325 — 19 files

👤 Participant 2462 — 1 sessions — 65 files total
    📂 2025-6-4_174114 — 65 files

👤 Participant 2973 — 1 sessions — 32 files total
    📂 2025-6-7_17131 — 32 files

👤 Participant 3154 — 2 sessions — 32 files total
    📂 2025-6-4_131612 — 30 files
    📂 2025-6-4_142625 — 2 files

👤 Participant 3291 — 2 sessions — 31 files total
    📂 2025-6-6_9201 — 30 files
    📂 2025-6-6_9957 — 1 files

👤 Participant 3709 — 2 sessions — 32 files total
    📂 2025-6-4_10021 — 31 files
    📂 2025-6-4_83728 — 1 files

👤 Participant 3814 — 2 sessions — 1

In [2]:
import os
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob

ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Adjust as needed

participant_data = defaultdict(list)

# Step 1: Traverse location folders (Dublin, Lisbon)
for location in os.listdir(ROOT):
    location_path = os.path.join(ROOT, location)
    if not os.path.isdir(location_path):
        continue

    # Step 2: Traverse participant folders within each location
    for participant_id in os.listdir(location_path):
        participant_path = os.path.join(location_path, participant_id)
        if not os.path.isdir(participant_path):
            continue

        heat_logs_path = os.path.join(participant_path, "heat_logs")
        if not os.path.exists(heat_logs_path):
            continue

        # Step 3: Traverse session folders
        for session_name in os.listdir(heat_logs_path):
            session_path = os.path.join(heat_logs_path, session_name)
            if not os.path.isdir(session_path):
                continue

            csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
            if not csv_files:
                print(f"⚠️ No CSV files in {participant_id}/{session_name}")
                continue

            session_dfs = []
            for file in csv_files:
                file_path = os.path.join(session_path, file)
                try:
                    df = pd.read_csv(file_path)
                    session_dfs.append(df)
                except Exception as e:
                    print(f"❌ Failed to read {file_path}: {e}")

            if session_dfs:
                session_df = pd.concat(session_dfs, ignore_index=True)
                participant_data[participant_id].append(session_df)

print(f"✅ Loaded data for {len(participant_data)} participants across all locations.")


⚠️ No CSV files in 1629/2025-6-23_14515
⚠️ No CSV files in 2609/2025-6-25_9643
⚠️ No CSV files in 4431/2025-6-24_10537
⚠️ No CSV files in 5564/2025-6-25_11949
⚠️ No CSV files in 6594/2025-6-26_111743
⚠️ No CSV files in 6758/2025-6-25_8751
⚠️ No CSV files in 7487/2025-6-24_11539
⚠️ No CSV files in 7487/2025-6-24_1166
⚠️ No CSV files in 7487/2025-6-24_132144
⚠️ No CSV files in 7487/2025-6-24_142936
⚠️ No CSV files in 7487/2025-6-24_144958
⚠️ No CSV files in 8851/2025-6-26_104939
⚠️ No CSV files in 8851/2025-6-26_105046
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files in 5985/2025-6-6_142335
⚠️ No CSV files in 9084/2025-6-7_13249
⚠

In [3]:
import os
import pandas as pd
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"  # Update if needed

# Output dictionary: key = participant ID, value = list of session DataFrames
participant_data = defaultdict(list)

# Optional: if you want to track where each participant belongs
participant_locations = {}

print("🔍 Scanning participants under each location (Dublin, Lisbon)...")

# Traverse top-level locations
for location in os.listdir(ROOT):
    location_path = os.path.join(ROOT, location)
    if not os.path.isdir(location_path):
        continue

    for participant_id in os.listdir(location_path):
        participant_path = os.path.join(location_path, participant_id)
        if not os.path.isdir(participant_path):
            continue

        heat_logs_path = os.path.join(participant_path, "heat_logs")
        if not os.path.exists(heat_logs_path):
            continue

        # Store participant's location
        participant_locations[participant_id] = location

        for session_name in os.listdir(heat_logs_path):
            session_path = os.path.join(heat_logs_path, session_name)
            if not os.path.isdir(session_path):
                continue

            csv_files = [f for f in os.listdir(session_path) if f.endswith(".csv")]
            if not csv_files:
                print(f"⚠️ No CSV files in {participant_id}/{session_name}")
                continue

            session_dfs = []
            for file in csv_files:
                file_path = os.path.join(session_path, file)
                try:
                    df = pd.read_csv(file_path)
                    session_dfs.append(df)
                except Exception as e:
                    print(f"❌ Failed to read {file_path}: {e}")

            if session_dfs:
                combined_df = pd.concat(session_dfs, ignore_index=True)
                participant_data[participant_id].append(combined_df)

print(f"\n✅ Loaded data for {len(participant_data)} participants.")


🔍 Scanning participants under each location (Dublin, Lisbon)...
⚠️ No CSV files in 1629/2025-6-23_14515
⚠️ No CSV files in 2609/2025-6-25_9643
⚠️ No CSV files in 4431/2025-6-24_10537
⚠️ No CSV files in 5564/2025-6-25_11949
⚠️ No CSV files in 6594/2025-6-26_111743
⚠️ No CSV files in 6758/2025-6-25_8751
⚠️ No CSV files in 7487/2025-6-24_11539
⚠️ No CSV files in 7487/2025-6-24_1166
⚠️ No CSV files in 7487/2025-6-24_132144
⚠️ No CSV files in 7487/2025-6-24_142936
⚠️ No CSV files in 7487/2025-6-24_144958
⚠️ No CSV files in 8851/2025-6-26_104939
⚠️ No CSV files in 8851/2025-6-26_105046
⚠️ No CSV files in 2150/2025-6-5_101933
⚠️ No CSV files in 2150/2025-6-5_102122
⚠️ No CSV files in 2150/2025-6-5_13161
⚠️ No CSV files in 3709/2025-6-4_83728
⚠️ No CSV files in 4050/2025-6-5_152940
⚠️ No CSV files in 4050/2025-6-5_153025
⚠️ No CSV files in 5390/2025-6-5_124511
⚠️ No CSV files in 5390/2025-6-5_124612
⚠️ No CSV files in 5390/2025-6-5_124740
⚠️ No CSV files in 5390/2025-6-5_12527
⚠️ No CSV files 

In [4]:
import os
from collections import defaultdict

# === CONFIGURATION ===
ROOT = r"C:\Users\bmoha\Desktop\DataCollection"

# === STATS STORAGE ===
location_participants = defaultdict(list)
participant_file_summary = defaultdict(lambda: {"location": "", "total_files": 0, "sessions": {}})

print("🔍 Scanning participant folders grouped by location...\n")

# === Step 1: Traverse Dublin and Lisbon folders ===
for location in os.listdir(ROOT):
    location_path = os.path.join(ROOT, location)
    if not os.path.isdir(location_path):
        continue

    for participant_id in os.listdir(location_path):
        participant_path = os.path.join(location_path, participant_id)
        if not os.path.isdir(participant_path):
            continue

        heat_logs_path = os.path.join(participant_path, "heat_logs")
        if not os.path.exists(heat_logs_path):
            continue

        # Track location for the participant
        location_participants[location].append(participant_id)
        participant_file_summary[participant_id]["location"] = location

        # === Step 2: Traverse sessions in heat_logs ===
        for session_name in os.listdir(heat_logs_path):
            session_path = os.path.join(heat_logs_path, session_name)
            if not os.path.isdir(session_path):
                continue

            csv_count = len([f for f in os.listdir(session_path) if f.endswith(".csv")])
            if csv_count > 0:
                participant_file_summary[participant_id]["total_files"] += csv_count
                participant_file_summary[participant_id]["sessions"][session_name] = csv_count

# === Step 3: Print Summary ===

# -- Total participants per location --
print("📊 Total Participants:")
for loc in sorted(location_participants.keys()):
    print(f"  🌍 {loc}: {len(location_participants[loc])} participants")

# -- Per participant breakdown --
print("\n📦 Participant File Summary:")
for pid, info in sorted(participant_file_summary.items()):
    print(f"\n👤 Participant {pid} ({info['location']}) — {len(info['sessions'])} sessions — {info['total_files']} files total")
    for session, count in sorted(info["sessions"].items()):
        print(f"    📂 {session} — {count} files")


🔍 Scanning participant folders grouped by location...

📊 Total Participants:
  🌍 Dublin: 36 participants
  🌍 Lisbon: 24 participants

📦 Participant File Summary:

👤 Participant 1060 (Lisbon) — 1 sessions — 65 files total
    📂 2025-6-5_13161 — 65 files

👤 Participant 1455 (Dublin) — 1 sessions — 62 files total
    📂 2025-6-25_155732 — 62 files

👤 Participant 1538 (Dublin) — 1 sessions — 62 files total
    📂 2025-6-28_10589 — 62 files

👤 Participant 1626 (Dublin) — 1 sessions — 61 files total
    📂 2025-6-26_151116 — 61 files

👤 Participant 1629 (Dublin) — 6 sessions — 114 files total
    📂 2025-6-23_144258 — 1 files
    📂 2025-6-23_144754 — 2 files
    📂 2025-6-23_145726 — 2 files
    📂 2025-6-23_1672 — 43 files
    📂 2025-6-24_111113 — 65 files
    📂 2025-6-24_1183 — 1 files

👤 Participant 1700 (Dublin) — 2 sessions — 41 files total
    📂 2025-6-26_141414 — 27 files
    📂 2025-6-26_144239 — 14 files

👤 Participant 1789 (Dublin) — 1 sessions — 62 files total
    📂 2025-6-27_101058 — 62